In [1]:
import pandas as pd
import numpy as np
import wbgapi as wb
import yfinance as yf

import statsmodels.api as sm
from statsmodels.formula.api import ols, logit

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

# Question 1

For this question use the World Bank Data for Turkey for the following indicators. Use [wbgapi](https://pypi.org/project/wbgapi/) for getting the data.

* [Literacy rate, adult female (SE.ADT.LITR.FE.ZS)](https://data.worldbank.org/indicator/SE.ADT.LITR.FE.ZS)
* [Labor force, female (SL.TLF.TOTL.FE.ZS)](https://data.worldbank.org/indicator/SL.TLF.TOTL.FE.ZS)
* [Poverty headcount ratio at national poverty lines (SI.POV.NAHC)](https://data.worldbank.org/indicator/SI.POV.NAHC)
* [Current health expenditure per capita (SH.XPD.CHEX.PC.CD)](https://data.worldbank.org/indicator/SH.XPD.CHEX.PC.CD)
* [GDP per capita (NY.GDP.PCAP.CD)](https://data.worldbank.org/indicator/NY.GDP.PCAP.CD)
* [Mortality rate, under-5 (SH.DYN.MORT)](https://data.worldbank.org/indicator/SH.DYN.MORT)


Using the [statsmodels](https://www.statsmodels.org/stable/index.html) library write the best linear regression model using child mortality as the dependent variable while the rest are considered as independent variables. Pay particular attention to the fact that the order of the variables put into the model significantly impacts the performance of the model. Choose the best model by considering

* with the minimum number of variables and their interactions,
* with the optimal ordering of the independent variables and their interactions,
* $R^2$-score of the model,
* statistical significance of the model coefficients,
* ANOVA analysis of the model.


## Answers for Q1

You can download more than one series specific to a country as follows:

In [2]:
data = wb.data.DataFrame(series = ['SE.ADT.LITR.FE.ZS','SL.TLF.TOTL.FE.ZS','SI.POV.NAHC',
                                   'SH.XPD.CHEX.PC.CD','NY.GDP.PCAP.CD','SH.DYN.MORT'],
                         economy='TUR').T
data.rename(columns={'SE.ADT.LITR.FE.ZS':'literacy',
                     'SL.TLF.TOTL.FE.ZS':'labor',
                     'SI.POV.NAHC':'poverty',
                     'SH.XPD.CHEX.PC.CD':'expenditure',
                     'NY.GDP.PCAP.CD':'gdp',
                     'SH.DYN.MORT':'mortality'}, inplace=True)
data.dropna(inplace=True)
data

series,gdp,literacy,mortality,expenditure,poverty,labor
YR2005,7456.296101,80.548950,26.2,364.908051,18.6,25.921956
YR2006,8101.856924,80.385162,24.3,416.928528,16.3,26.205298
YR2007,9791.882450,81.263832,22.5,512.848267,16.7,26.288681
YR2009,9103.474051,85.347160,19.5,500.193054,16.9,28.071295
YR2010,10742.774979,88.073174,18.1,539.327148,16.1,29.161917
YR2011,11420.555456,90.310097,16.9,531.418579,16.3,29.832127
YR2012,11795.633457,91.604523,15.8,524.250305,15.0,30.464131
YR2013,12614.781610,92.138077,14.7,551.401245,15.0,31.199391
YR2014,12157.990434,92.401817,13.8,525.844727,14.7,31.027784
YR2015,11006.279524,92.645813,13.0,453.116486,14.3,31.719798


The first thing you must do before you do a linear regression problem is to check if independent variables are correlated. Because high correlation among the independent variables degredates the quality of the resulting model.

In [3]:
data.corr()

series,gdp,literacy,mortality,expenditure,poverty,labor
series,,,,,,
gdp,1.000000,0.708098,-0.624924,0.786933,-0.642485,0.591484
literacy,0.708098,1.000000,-0.976415,0.240982,-0.849316,0.981657
mortality,-0.624924,-0.976415,1.000000,-0.173542,0.885762,-0.989861
expenditure,0.786933,0.240982,-0.173542,1.000000,-0.192544,0.097192
poverty,-0.642485,-0.849316,0.885762,-0.192544,1.000000,-0.867518
labor,0.591484,0.981657,-0.989861,0.097192,-0.867518,1.000000


Female labor partipication, female literacy rate and child mortality are highly correlated. Since our dependent variable is mortality, it is better if dropped one of literacy or labor participation. I'll keep labor partipation.

In [4]:
model = ols('mortality ~  poverty + expenditure + labor + gdp', data=data).fit()
model.summary()

/home/kaygun/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              mortality   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     242.2
Date:                Thu, 17 Nov 2022   Prob (F-statistic):           2.23e-08
Time:                        20:27:44   Log-Likelihood:                -7.8525
No. Observations:                  13   AIC:                             25.70
Df Residuals:                       8   BIC:                             28.53
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      70.1838      7.670      9.151      0.000      52.497      87.870
poverty         0.4428      0.249      1.777      0.113      -0.132       1.017
expenditure    -0.0185      0.007     -2.783      0.024      -0.034      -0.003
labor          -1.9570      0.146    -13.445      0.000      -2.293      -1.621
gdp             0.0007      0.000      2.092      0.070   -6.96e-05       0.001
==============================================================================
Omnibus:                        1.741   Durbin-Watson:                   2.147
Prob(Omnibus):                  0.419   Jarque-Bera (JB):                1.157
Skew:                          -0.484   Prob(JB):                        0.561
Kurtosis:                       1.905   Cond. No.                     5.14e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.14e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Among the model coefficients only expenditure and labor are significant. But before we settle on our model, let us check the ANOVA table:

In [5]:
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
poverty,1.0,244.065138,244.065138,766.422629,3.126859e-09
expenditure,1.0,0.002896,0.002896,0.009093,9.263777e-01
labor,1.0,63.070572,63.070572,198.056610,6.311465e-07
gdp,1.0,1.393817,1.393817,4.376917,6.978890e-02
Residual,8.0,2.547578,0.318447,NaN,NaN


It seems poverty seems to be important along with labor. But recall that the ordering of the varibles is also important:

In [6]:
model = ols('mortality ~ labor + poverty', data=data).fit()
print(model.summary())
sm.stats.anova_lm(model)

                            OLS Regression Results                            
Dep. Variable:              mortality   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     285.4
Date:                Thu, 17 Nov 2022   Prob (F-statistic):           1.51e-09
Time:                        20:27:44   Log-Likelihood:                -12.683
No. Observations:                  13   AIC:                             31.37
Df Residuals:                      10   BIC:                             33.06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     63.7616      9.301      6.855      0.0

/home/kaygun/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


,df,sum_sq,mean_sq,F,PR(>F)
labor,1.0,304.804040,304.804040,569.016826,3.808848e-10
poverty,1.0,0.919281,0.919281,1.716140,2.194839e-01
Residual,10.0,5.356679,0.535668,NaN,NaN


The coefficent of the poverty variable still isn't significant. Moreover, when we changed the order, the contribution of the variable poverty to the variance has dropped significantly. So, one variable seems to enough:

In [7]:
model = ols('mortality ~ labor', data=data).fit()
print(model.summary())
sm.stats.anova_lm(model)

                            OLS Regression Results                            
Dep. Variable:              mortality   R-squared:                       0.980
Model:                            OLS   Adj. R-squared:                  0.978
Method:                 Least Squares   F-statistic:                     534.2
Date:                Thu, 17 Nov 2022   Prob (F-statistic):           1.13e-10
Time:                        20:27:44   Log-Likelihood:                -13.713
No. Observations:                  13   AIC:                             31.43
Df Residuals:                      11   BIC:                             32.56
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     75.5089      2.549     29.627      0.0

/home/kaygun/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


,df,sum_sq,mean_sq,F,PR(>F)
labor,1.0,304.80404,304.804040,534.236098,1.126086e-10
Residual,11.0,6.27596,0.570542,NaN,NaN


Our analysis indicates that as the female participation to the labor force increases the child mortality drops. In other words, the more women work outside of the homes, the lesser the child mortality. Also, female literacy rate and female labor participation are highly correlated. The analysis does not indicate a cause-effect relationship between literacy rates and labor participation for women. But whichever is the driver, both have a negative relationship with child mortality. The more women learn reading and writing and work outside homes, the lesser the child mortality rates.

# Question 2

For this question use Yahoo's Finance API for the following tickers:

* Gold futures (GC=F)
* Silver futures (SI=F)
* Copper futures (HG=F)
* Platinum futures (PL=F)

1. Write the best linear regression model that explains gold futures closing prices in terms of opening prices of gold, silver, copper, and platinum futures.
2. Repeat the same for silver, copper and platinum prices.
3. Compare the models you obtained in Steps 1 and 2. Which model is better? How do you decide? Explain.

## Answers for Q2

In [8]:
futures = yf.download(['GC=F','SI=F','HG=F','PL=F'])
futures

[*********************100%***********************]  4 of 4 completed


Adj Close                                        Close          \
                   GC=F    HG=F         PL=F       SI=F         GC=F    HG=F   
Date                                                                           
2000-01-04          NaN     NaN   429.700012        NaN          NaN     NaN   
2000-01-05          NaN     NaN   419.899994        NaN          NaN     NaN   
2000-01-06          NaN     NaN   412.000000        NaN          NaN     NaN   
2000-01-07          NaN     NaN   414.000000        NaN          NaN     NaN   
2000-01-10          NaN     NaN   415.000000        NaN          NaN     NaN   
...                 ...     ...          ...        ...          ...     ...   
2022-11-11  1766.000000  3.9510  1047.400024  21.646000  1766.000000  3.9510   
2022-11-14  1773.599976  3.8550  1042.300049  22.091999  1773.599976  3.8550   
2022-11-15  1773.800049  3.8395  1031.699951  21.499001  1773.800049  3.8395   
2022-11-16  1773.000000  3.7880  1025.500000  21.504000  1773.000000  3.7880   
2022-11-17  1760.400024  3.6840   987.700012  20.940001  1760.400024  3.6840   

                                           High          ...          Low  \
                   PL=F       SI=F         GC=F    HG=F  ...         PL=F   
Date                                                     ...                
2000-01-04   429.700012        NaN          NaN     NaN  ...   421.000000   
2000-01-05   419.899994        NaN          NaN     NaN  ...   416.899994   
2000-01-06   412.000000        NaN          NaN     NaN  ...   409.000000   
2000-01-07   414.000000        NaN          NaN     NaN  ...   408.000000   
2000-01-10   415.000000        NaN          NaN     NaN  ...   414.000000   
...                 ...        ...          ...     ...  ...          ...   
2022-11-11  1047.400024  21.646000  1766.000000  3.9510  ...  1047.400024   
2022-11-14  1042.300049  22.091999  1773.599976  3.8720  ...  1042.300049   
2022-11-15  1031.699951  21.499001  1773.800049  3.8505  ...  1031.699951   
2022-11-16  1025.500000  21.504000  1778.900024  3.7880  ...  1025.500000   
2022-11-17   987.700012  20.940001  1777.599976  3.7725  ...   985.000000   

                              Open                                   Volume  \
                 SI=F         GC=F   HG=F         PL=F       SI=F      GC=F   
Date                                                                          
2000-01-04        NaN          NaN    NaN   428.000000        NaN       NaN   
2000-01-05        NaN          NaN    NaN   428.000000        NaN       NaN   
2000-01-06        NaN          NaN    NaN   415.000000        NaN       NaN   
2000-01-07        NaN          NaN    NaN   408.000000        NaN       NaN   
2000-01-10        NaN          NaN    NaN   414.000000        NaN       NaN   
...               ...          ...    ...          ...        ...       ...   
2022-11-11  21.514999  1763.800049  3.917  1047.400024  21.584999     257.0   
2022-11-14  21.855000  1762.000000  3.848  1042.300049  21.865000     358.0   
2022-11-15  21.450001  1770.800049  3.849  1031.699951  22.030001      57.0   
2022-11-16  21.504000  1776.599976  3.788  1025.500000  21.625000      57.0   
2022-11-17  20.785000  1777.000000  3.761  1016.000000  21.504999  141461.0   

                                       
               HG=F     PL=F     SI=F  
Date                                   
2000-01-04      NaN    632.0      NaN  
2000-01-05      NaN    571.0      NaN  
2000-01-06      NaN    494.0      NaN  
2000-01-07      NaN    130.0      NaN  
2000-01-10      NaN     24.0      NaN  
...             ...      ...      ...  
2022-11-11    260.0      1.0     71.0  
2022-11-14    661.0      1.0     13.0  
2022-11-15    409.0      1.0     32.0  
2022-11-16    409.0      1.0     32.0  
2022-11-17  46793.0  15974.0  54903.0  

[5750 rows x 24 columns]

In [9]:
data = futures[['Open','Close']].dropna()
data

Open                                       Close          \
                   GC=F   HG=F         PL=F       SI=F         GC=F    HG=F   
Date                                                                          
2000-08-30   273.899994  0.879   593.900024   4.950000   273.899994  0.8850   
2000-08-31   274.799988  0.885   589.000000   4.920000   278.299988  0.8850   
2000-09-01   277.000000  0.878   588.000000   5.035000   277.000000  0.8890   
2000-09-05   275.799988  0.896   602.000000   4.990000   275.799988  0.9060   
2000-09-06   274.200012  0.905   603.000000   5.000000   274.200012  0.9015   
...                 ...    ...          ...        ...          ...     ...   
2022-11-11  1763.800049  3.917  1047.400024  21.584999  1766.000000  3.9510   
2022-11-14  1762.000000  3.848  1042.300049  21.865000  1773.599976  3.8550   
2022-11-15  1770.800049  3.849  1031.699951  22.030001  1773.800049  3.8395   
2022-11-16  1776.599976  3.788  1025.500000  21.625000  1773.000000  3.7880   
2022-11-17  1777.000000  3.761  1016.000000  21.504999  1760.400024  3.6840   

                                    
                   PL=F       SI=F  
Date                                
2000-08-30   591.400024   4.930000  
2000-08-31   586.700012   5.003000  
2000-09-01   595.299988   5.004000  
2000-09-05   601.299988   4.998000  
2000-09-06   611.099976   4.983000  
...                 ...        ...  
2022-11-11  1047.400024  21.646000  
2022-11-14  1042.300049  22.091999  
2022-11-15  1031.699951  21.499001  
2022-11-16  1025.500000  21.504000  
2022-11-17   987.700012  20.940001  

[4873 rows x 8 columns]

In [10]:
data.columns = ['gold_open','copper_open','platinum_open','silver_open',
                'gold_close','copper_close','platinum_close','silver_close']
data

,gold_open,copper_open,platinum_open,silver_open,gold_close,copper_close,platinum_close,silver_close
Date,,,,,,,,
2000-08-30,273.899994,0.879,593.900024,4.950000,273.899994,0.8850,591.400024,4.930000
2000-08-31,274.799988,0.885,589.000000,4.920000,278.299988,0.8850,586.700012,5.003000
2000-09-01,277.000000,0.878,588.000000,5.035000,277.000000,0.8890,595.299988,5.004000
2000-09-05,275.799988,0.896,602.000000,4.990000,275.799988,0.9060,601.299988,4.998000
2000-09-06,274.200012,0.905,603.000000,5.000000,274.200012,0.9015,611.099976,4.983000
...,...,...,...,...,...,...,...,...
2022-11-11,1763.800049,3.917,1047.400024,21.584999,1766.000000,3.9510,1047.400024,21.646000
2022-11-14,1762.000000,3.848,1042.300049,21.865000,1773.599976,3.8550,1042.300049,22.091999
2022-11-15,1770.800049,3.849,1031.699951,22.030001,1773.800049,3.8395,1031.699951,21.499001


In [11]:
data.corr()

,gold_open,copper_open,platinum_open,silver_open,gold_close,copper_close,platinum_close,silver_close
gold_open,1.000000,0.830371,0.500505,0.869802,0.999758,0.829660,0.499915,0.869478
copper_open,0.830371,1.000000,0.743147,0.854540,0.830407,0.999411,0.742731,0.854649
platinum_open,0.500505,0.743147,1.000000,0.760726,0.500397,0.743056,0.999707,0.761109
silver_open,0.869802,0.854540,0.760726,1.000000,0.869586,0.853605,0.760366,0.999327
gold_close,0.999758,0.830407,0.500397,0.869586,1.000000,0.829864,0.499939,0.869739
copper_close,0.829660,0.999411,0.743056,0.853605,0.829864,1.000000,0.742788,0.854082
platinum_close,0.499915,0.742731,0.999707,0.760366,0.499939,0.742788,1.000000,0.760897
silver_close,0.869478,0.854649,0.761109,0.999327,0.869739,0.854082,0.760897,1.000000


Gold futures closing price is highly correlated with its opening price. Silver and copper opening prices also show high correlation with gold futures closing price. However, it also seems that platinum opening prices are not strongly related with gold futures closing prices. Let us write our model.

In [12]:
model = ols('gold_close ~ gold_open + copper_open + silver_open + platinum_open', data=data).fit()
print(model.summary())
sm.stats.anova_lm(model)

                            OLS Regression Results                            
Dep. Variable:             gold_close   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.516e+06
Date:                Thu, 17 Nov 2022   Prob (F-statistic):               0.00
Time:                        20:27:45   Log-Likelihood:                -18767.
No. Observations:                4873   AIC:                         3.754e+04
Df Residuals:                    4868   BIC:                         3.758e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.4537      0.661      0.687

,df,sum_sq,mean_sq,F,PR(>F)
gold_open,1.0,1.305421e+09,1.305421e+09,1.006293e+07,0.000000
copper_open,1.0,2.358927e+02,2.358927e+02,1.818396e+00,0.177567
silver_open,1.0,7.869381e+01,7.869381e+01,6.066171e-01,0.436102
platinum_open,1.0,1.023025e+02,1.023025e+02,7.886063e-01,0.374566
Residual,4868.0,6.315046e+05,1.297257e+02,NaN,NaN


ANOVA analysis seem to indicate opening prices for gold futures is enough for model building. 

In [13]:
model = ols('gold_close ~ gold_open', data=data).fit()
print(model.summary())
sm.stats.anova_lm(model)

                            OLS Regression Results                            
Dep. Variable:             gold_close   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.006e+07
Date:                Thu, 17 Nov 2022   Prob (F-statistic):               0.00
Time:                        20:27:45   Log-Likelihood:                -18768.
No. Observations:                4873   AIC:                         3.754e+04
Df Residuals:                    4871   BIC:                         3.755e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3047      0.388      0.785      0.4

,df,sum_sq,mean_sq,F,PR(>F)
gold_open,1.0,1.305421e+09,1.305421e+09,1.006249e+07,0.0
Residual,4871.0,6.319214e+05,1.297314e+02,NaN,NaN


Let us repeat this for other futures. Let us start with silver and write the largest model possible with all interactions.

In [14]:
model = ols('silver_close ~ silver_open * gold_open * platinum_open * copper_open', data=data).fit()
print(model.summary())
sm.stats.anova_lm(model)

                            OLS Regression Results                            
Dep. Variable:           silver_close   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 2.448e+05
Date:                Thu, 17 Nov 2022   Prob (F-statistic):               0.00
Time:                        20:27:46   Log-Likelihood:                -1276.9
No. Observations:                4873   AIC:                             2586.
Df Residuals:                    4857   BIC:                             2690.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------

,df,sum_sq,mean_sq,F,PR(>F)
silver_open,1.0,364233.242791,364233.242791,3.671311e+06,0.000000e+00
gold_open,1.0,0.102638,0.102638,1.034542e+00,3.091449e-01
silver_open:gold_open,1.0,0.612727,0.612727,6.176016e+00,1.298276e-02
platinum_open,1.0,0.711787,0.711787,7.174501e+00,7.419605e-03
silver_open:platinum_open,1.0,0.890857,0.890857,8.979445e+00,2.744139e-03
gold_open:platinum_open,1.0,0.372490,0.372490,3.754534e+00,5.272237e-02
silver_open:gold_open:platinum_open,1.0,0.004975,0.004975,5.014306e-02,8.228239e-01
copper_open,1.0,0.600271,0.600271,6.050465e+00,1.393723e-02
silver_open:copper_open,1.0,0.550935,0.550935,5.553183e+00,1.848660e-02
gold_open:copper_open,1.0,1.317554,1.317554,1.328037e+01,2.710028e-04


As before, the opening prices were enough.

In [15]:
model = ols('silver_close ~ silver_open', data=data).fit()
print(model.summary())
sm.stats.anova_lm(model)

                            OLS Regression Results                            
Dep. Variable:           silver_close   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 3.614e+06
Date:                Thu, 17 Nov 2022   Prob (F-statistic):               0.00
Time:                        20:27:46   Log-Likelihood:                -1322.1
No. Observations:                4873   AIC:                             2648.
Df Residuals:                    4871   BIC:                             2661.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0200      0.010      2.007      

,df,sum_sq,mean_sq,F,PR(>F)
silver_open,1.0,364233.242791,364233.242791,3.614263e+06,0.0
Residual,4871.0,490.883018,0.100777,NaN,NaN


Next, copper prices:

In [16]:
model = ols('copper_close ~ copper_open * gold_open * platinum_open * silver_open', data=data).fit()
print(model.summary())
sm.stats.anova_lm(model)

                            OLS Regression Results                            
Dep. Variable:           copper_close   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 2.762e+05
Date:                Thu, 17 Nov 2022   Prob (F-statistic):               0.00
Time:                        20:27:46   Log-Likelihood:                 9011.9
No. Observations:                4873   AIC:                        -1.799e+04
Df Residuals:                    4857   BIC:                        -1.789e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------

,df,sum_sq,mean_sq,F,PR(>F)
copper_open,1.0,6023.636556,6023.636556,4.142227e+06,0.000000
gold_open,1.0,0.000955,0.000955,6.566493e-01,0.417785
copper_open:gold_open,1.0,0.005391,0.005391,3.706893e+00,0.054246
platinum_open,1.0,0.000234,0.000234,1.612334e-01,0.688041
copper_open:platinum_open,1.0,0.007688,0.007688,5.286683e+00,0.021531
gold_open:platinum_open,1.0,0.011402,0.011402,7.841041e+00,0.005128
copper_open:gold_open:platinum_open,1.0,0.000644,0.000644,4.431435e-01,0.505641
silver_open,1.0,0.000910,0.000910,6.259947e-01,0.428867
copper_open:silver_open,1.0,0.005315,0.005315,3.654887e+00,0.055964
gold_open:silver_open,1.0,0.000055,0.000055,3.788117e-02,0.845690


In [17]:
model = ols('copper_close ~ copper_open', data=data).fit()
print(model.summary())
sm.stats.anova_lm(model)

                            OLS Regression Results                            
Dep. Variable:           copper_close   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 4.128e+06
Date:                Thu, 17 Nov 2022   Prob (F-statistic):               0.00
Time:                        20:27:46   Log-Likelihood:                 8996.6
No. Observations:                4873   AIC:                        -1.799e+04
Df Residuals:                    4871   BIC:                        -1.798e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0024      0.001      1.661      

,df,sum_sq,mean_sq,F,PR(>F)
copper_open,1.0,6023.636556,6023.636556,4.128123e+06,0.0
Residual,4871.0,7.107622,0.001459,NaN,NaN


And finally for platinum prices:

In [18]:
model = ols('platinum_close ~ platinum_open * gold_open * copper_open * silver_open', data=data).fit()
print(model.summary())
sm.stats.anova_lm(model)

                            OLS Regression Results                            
Dep. Variable:         platinum_close   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 5.590e+05
Date:                Thu, 17 Nov 2022   Prob (F-statistic):               0.00
Time:                        20:27:46   Log-Likelihood:                -17514.
No. Observations:                4873   AIC:                         3.506e+04
Df Residuals:                    4857   BIC:                         3.516e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------

,df,sum_sq,mean_sq,F,PR(>F)
platinum_open,1.0,6.519600e+08,6.519600e+08,8.384801e+06,0.000000
gold_open,1.0,1.713511e+02,1.713511e+02,2.203732e+00,0.137742
platinum_open:gold_open,1.0,1.309275e+02,1.309275e+02,1.683848e+00,0.194476
copper_open,1.0,1.390985e-01,1.390985e-01,1.788934e-03,0.966265
platinum_open:copper_open,1.0,1.307907e+02,1.307907e+02,1.682087e+00,0.194709
gold_open:copper_open,1.0,3.259275e+01,3.259275e+01,4.191725e-01,0.517381
platinum_open:gold_open:copper_open,1.0,6.734063e+01,6.734063e+01,8.660619e-01,0.352094
silver_open,1.0,1.839719e+03,1.839719e+03,2.366046e+01,0.000001
platinum_open:silver_open,1.0,1.620135e+02,1.620135e+02,2.083641e+00,0.148949
gold_open:silver_open,1.0,1.526192e+02,1.526192e+02,1.962822e+00,0.161276


In [19]:
model = ols('platinum_close ~ platinum_open + gold_open + silver_open', data=data).fit()
print(model.summary())
sm.stats.anova_lm(model)

                            OLS Regression Results                            
Dep. Variable:         platinum_close   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 2.774e+06
Date:                Thu, 17 Nov 2022   Prob (F-statistic):               0.00
Time:                        20:27:46   Log-Likelihood:                -17538.
No. Observations:                4873   AIC:                         3.508e+04
Df Residuals:                    4869   BIC:                         3.511e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.8189      0.505      1.621

,df,sum_sq,mean_sq,F,PR(>F)
platinum_open,1.0,6.519600e+08,6.519600e+08,8.323082e+06,0.000000
gold_open,1.0,1.713511e+02,1.713511e+02,2.187510e+00,0.139199
silver_open,1.0,1.483157e+02,1.483157e+02,1.893434e+00,0.168878
Residual,4869.0,3.813964e+05,7.833156e+01,NaN,NaN


# Question 3

1. Write a function that takes a ticker symbol and returns a pandas dataframe that for each day puts a 1 when the closing price is higher than the opening price, a 0 when the closing price is lower than the opening price.
2. Write the best logistic regression that predicts the time series you obtain from Step 1 for gold futures against the opening prices of gold, silver, copper, and platinum prices.
3. Repeat the same for silver, copper, and platinum prices.
4. Compare the models you obtained from Steps 2 and 3. Decide which is the best model, and explain your reasoning.
5. Does any of the models provide a good fit? Explain.

In [20]:
def isClosingHiger(ticker):
    tmp = yf.download(ticker)[['Open','Close']]
    fn = {True: 1, False: 0}
    return (tmp ['Close'] >= tmp['Open']).replace({False: 0, True: 1})

In [21]:
data['gold'] = isClosingHiger('GC=F')
data['silver'] = isClosingHiger('SI=F')
data['platinum'] = isClosingHiger('PL=F')
data['copper'] = isClosingHiger('HG=F')
data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,gold_open,copper_open,platinum_open,silver_open,gold_close,copper_close,platinum_close,silver_close,gold,silver,platinum,copper
Date,,,,,,,,,,,,
2000-08-30,273.899994,0.879,593.900024,4.950000,273.899994,0.8850,591.400024,4.930000,1,0,0,1
2000-08-31,274.799988,0.885,589.000000,4.920000,278.299988,0.8850,586.700012,5.003000,1,1,0,1
2000-09-01,277.000000,0.878,588.000000,5.035000,277.000000,0.8890,595.299988,5.004000,1,0,1,1
2000-09-05,275.799988,0.896,602.000000,4.990000,275.799988,0.9060,601.299988,4.998000,1,1,0,1
2000-09-06,274.200012,0.905,603.000000,5.000000,274.200012,0.9015,611.099976,4.983000,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-11,1763.800049,3.917,1047.400024,21.584999,1766.000000,3.9510,1047.400024,21.646000,1,1,1,1
2022-11-14,1762.000000,3.848,1042.300049,21.865000,1773.599976,3.8550,1042.300049,22.091999,1,1,1,1
2022-11-15,1770.800049,3.849,1031.699951,22.030001,1773.800049,3.8395,1031.699951,21.499001,1,0,1,0


In [24]:
model = logit("gold ~ gold_open * silver_open * copper_open * platinum_open", data=data).fit()
print(model.summary())
y_pred = model.predict(data)
confusion_matrix(data.gold,y_pred > 0.5)

Optimization terminated successfully.
         Current function value: 0.658416
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                   gold   No. Observations:                 4873
Model:                          Logit   Df Residuals:                     4857
Method:                           MLE   Df Model:                           15
Date:                Thu, 17 Nov 2022   Pseudo R-squ.:                 0.02426
Time:                        20:29:22   Log-Likelihood:                -3208.5
converged:                       True   LL-Null:                       -3288.2
Covariance Type:            nonrobust   LLR p-value:                 3.069e-26
                                                      coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
Intercept                         

array([[ 305, 1666],
       [ 326, 2576]])

In [25]:
model = logit("silver ~ gold_open * silver_open * copper_open * platinum_open", data=data).fit()
print(model.summary())
y_pred = model.predict(data)
confusion_matrix(data.silver,y_pred > 0.5)

Optimization terminated successfully.
         Current function value: 0.625962
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 silver   No. Observations:                 4873
Model:                          Logit   Df Residuals:                     4857
Method:                           MLE   Df Model:                           15
Date:                Thu, 17 Nov 2022   Pseudo R-squ.:                 0.03058
Time:                        20:29:36   Log-Likelihood:                -3050.3
converged:                       True   LL-Null:                       -3146.5
Covariance Type:            nonrobust   LLR p-value:                 7.239e-33
                                                      coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
Intercept                         

array([[  92, 1600],
       [  91, 3090]])

In [26]:
model = logit("copper ~ gold_open * silver_open * copper_open * platinum_open", data=data).fit()
print(model.summary())
y_pred = model.predict(data)
confusion_matrix(data.copper,y_pred > 0.5)

Optimization terminated successfully.
         Current function value: 0.685082
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                 copper   No. Observations:                 4873
Model:                          Logit   Df Residuals:                     4857
Method:                           MLE   Df Model:                           15
Date:                Thu, 17 Nov 2022   Pseudo R-squ.:                0.005700
Time:                        20:29:48   Log-Likelihood:                -3338.4
converged:                       True   LL-Null:                       -3357.5
Covariance Type:            nonrobust   LLR p-value:                 0.0008216
                                                      coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
Intercept                         

array([[ 283, 1932],
       [ 252, 2406]])

In [27]:
model = logit("platinum ~ gold_open * silver_open * copper_open * platinum_open", data=data).fit()
print(model.summary())
y_pred = model.predict(data)
confusion_matrix(data.platinum,y_pred > 0.5)

Optimization terminated successfully.
         Current function value: 0.502040
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               platinum   No. Observations:                 4873
Model:                          Logit   Df Residuals:                     4857
Method:                           MLE   Df Model:                           15
Date:                Thu, 17 Nov 2022   Pseudo R-squ.:                 0.06351
Time:                        20:29:58   Log-Likelihood:                -2446.4
converged:                       True   LL-Null:                       -2612.3
Covariance Type:            nonrobust   LLR p-value:                 1.326e-61
                                                      coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
Intercept                         

array([[   0, 1108],
       [   0, 3765]])

None of the models can predict reliably whether closing prices are going to be higher or lower than the opening prices. But in the previous question we saw that the closing prices are determined pretty accurately by the opening prices. Do the results of this question contradict with the previous question? Can you explain?

# Question 4

For this question use the following [data](https://archive.ics.uci.edu/ml/datasets/credit+approval):


In [28]:
credit = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', header=None)
X = credit.replace('?',0).iloc[:,[1,2,7,10,14]]
y = credit.iloc[:,15].replace({'+': 1, '-': 0})

1. Split the data into training and test set.
2. Write different logistic regression models predicting y against X.
3. Construct [confusion matrices](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) on the test data set for these different models.
4. Analyze these models. Explain which model is the best model you have found.
5. Repeat Steps 1-4 several times. Does your best model stay as the best model? What should be the correct protocol to decide on the best model explaining the data?

In [29]:
data = pd.DataFrame(X.astype(float))
data.columns = ['a','b','c','d','e']
data['y'] = y
data

,a,b,c,d,e,y
0,30.83,0.000,1.25,1.0,0.0,1
1,58.67,4.460,3.04,6.0,560.0,1
2,24.50,0.500,1.50,0.0,824.0,1
3,27.83,1.540,3.75,5.0,3.0,1
4,20.17,5.625,1.71,0.0,0.0,1
...,...,...,...,...,...,...
685,21.08,10.085,1.25,0.0,0.0,0
686,22.67,0.750,2.00,2.0,394.0,0
687,25.25,13.500,2.00,1.0,1.0,0
688,17.92,0.205,0.04,0.0,750.0,0


In [30]:
model = logit('y ~ a + b + c + d',data=data).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.521267
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  690
Model:                          Logit   Df Residuals:                      685
Method:                           MLE   Df Model:                            4
Date:                Thu, 17 Nov 2022   Pseudo R-squ.:                  0.2413
Time:                        20:30:09   Log-Likelihood:                -359.67
converged:                       True   LL-Null:                       -474.08
Covariance Type:            nonrobust   LLR p-value:                 2.385e-48
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.5940      0.259     -6.152      0.000      -2.102      -1.086
a              0.0051      0.008      0.653      0.514      -0.010       0.020
b              0.0390      0.020      1.999      0.046       0.001       0.077
c              0.1997      0.039      5.157      0.000       0.124       0.276
d              0.3627      0.043      8.481      0.000       0.279       0.447
==============================================================================
"""

The model can only explain 25% of the variation. This is the simplest model. If we add other interactions, pseudo-R2 does not change significantly. So, we will use this model.

In [31]:
def experiment(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.75)
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print(confusion_matrix(y_test,y_pred))
    print(model.score(X_test,y_test))

In [32]:
for i in range(10):
    experiment(X,y)

[[86 12]
 [29 46]]
0.7630057803468208
[[81 13]
 [27 52]]
0.7687861271676301
[[85 16]
 [23 49]]
0.7745664739884393
[[76 11]
 [37 49]]
0.7225433526011561
[[82  9]
 [34 48]]
0.7514450867052023
[[83 13]
 [34 43]]
0.7283236994219653
[[79 10]
 [35 49]]
0.7398843930635838
[[91  6]
 [30 46]]
0.791907514450867
[[89 11]
 [26 47]]
0.7861271676300579
[[84 11]
 [23 55]]
0.8034682080924855
